# Important run notebook #003

this is needed to populate the database with the stock data.

In [2]:
from pyspark.sql import SparkSession


database = "hacking"
collection = "stocks"
connectionString = "mongodb://127.0.0.1"

spark = SparkSession \
    .builder \
    .appName("dates") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.5') \
    .config("spark.mongodb.input.uri", connectionString) \
    .config("spark.mongodb.output.uri", connectionString) \
    .getOrCreate()


22/11/10 15:30:32 WARN Utils: Your hostname, mbp.local resolves to a loopback address: 127.0.0.1; using 192.168.18.6 instead (on interface en0)
22/11/10 15:30:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/tmillar/dev/repo/hacking-spark-with-mongo/venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tmillar/.ivy2/cache
The jars for the packages stored in: /Users/tmillar/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8a360749-ef3a-4371-971e-954ee3b8ff53;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.5 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 1665ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.5 from central in [default]
	---------------------------------------------------------------------
	|                 

22/11/10 15:30:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/10 15:30:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/10 15:30:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/10 15:30:35 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/10 15:30:35 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/11/10 15:30:35 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/11/10 15:30:35 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/11/10 15:30:35 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


In [3]:
df = spark.read.format("mongodb") \
    .option("database", database) \
    .option("collection", collection) \
    .load()

In [5]:
df.show()
df.printSchema()

+------------------+------------------+-------------------+------------------+------------------+------------------+---------+--------------------+
|         Adj Close|             Close|               Date|              High|               Low|              Open|   Volume|                 _id|
+------------------+------------------+-------------------+------------------+------------------+------------------+---------+--------------------+
|         27.727039|        214.009998|2010-01-04 00:00:00|        214.499996|212.38000099999996|        213.429998|123432400|636d5432962d673da...|
|27.774976000000002|        214.379993|2010-01-05 00:00:00|        215.589994|        213.249994|        214.599998|150476200|636d5432962d673da...|
|27.333178000000004|        210.969995|2010-01-06 00:00:00|            215.23|        210.750004|        214.379993|138040000|636d5432962d673da...|
|          27.28265|            210.58|2010-01-07 00:00:00|        212.000006|        209.050005|            211

Perform some functions on Date

In [11]:
from pyspark.sql.functions import (
    dayofmonth,
    dayofyear,
    hour,
    month,
    year,
    weekofyear,
    format_number,
    date_format
)

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [ ]:
df.select(
    dayofmonth(df["Date"])
).show()

In [12]:
df.select(
    dayofyear(df["Date"])
).show()

+---------------+
|dayofyear(Date)|
+---------------+
|              4|
|              5|
|              6|
|              7|
|              8|
|             11|
|             12|
|             13|
|             14|
|             15|
|             19|
|             20|
|             21|
|             22|
|             25|
|             26|
|             27|
|             28|
|             29|
|             32|
+---------------+
only showing top 20 rows



In [ ]:
df.select(
    hour(df["Date"])
).show()

In [49]:
df.select(
    date_format(df["Date"], "MMM,d yyyy").alias("Date")
).show()

+-----------+
|       Date|
+-----------+
| Jan,4 2010|
| Jan,5 2010|
| Jan,6 2010|
| Jan,7 2010|
| Jan,8 2010|
|Jan,11 2010|
|Jan,12 2010|
|Jan,13 2010|
|Jan,14 2010|
|Jan,15 2010|
|Jan,19 2010|
|Jan,20 2010|
|Jan,21 2010|
|Jan,22 2010|
|Jan,25 2010|
|Jan,26 2010|
|Jan,27 2010|
|Jan,28 2010|
|Jan,29 2010|
| Feb,1 2010|
+-----------+
only showing top 20 rows



Group by month then AVG Close

In [ ]:
df_month = df.select(
    month(df["Date"]).alias("month"),
    "Date",
    "Close"
)

df_month.groupBy("month").avg().select("month", "avg(Close)").show()

Group by Year then Max High

In [32]:
df_year = df.select(
    year(df["Date"]).alias("Year"),
    "Date",
    "High"
)

max_df = df_year.groupBy("Year").max().select("Year", "max(High)")

max_df.show()

sorted_df = max_df.orderBy(max_df["max(High)"].desc()).select("Year", max_df["max(High)"].alias("Max High"))

sorted_df.show()

formatted_df = sorted_df.select("Year", format_number(sorted_df["Max High"],3).alias("Max High"))

formatted_df.show()

+----+------------------+
|Year|         max(High)|
+----+------------------+
|2010|            326.66|
|2011|426.69999299999995|
|2012|        705.070023|
|2013|        575.139999|
|2014|        651.259979|
|2015|134.53999299999998|
|2016|        118.690002|
+----+------------------+

+----+------------------+
|Year|          Max High|
+----+------------------+
|2012|        705.070023|
|2014|        651.259979|
|2013|        575.139999|
|2011|426.69999299999995|
|2010|            326.66|
|2015|134.53999299999998|
|2016|        118.690002|
+----+------------------+

+----+--------+
|Year|Max High|
+----+--------+
|2012| 705.070|
|2014| 651.260|
|2013| 575.140|
|2011| 426.700|
|2010| 326.660|
|2015| 134.540|
|2016| 118.690|
+----+--------+



Read stocks CSV and infer schema then save to MongoDB

In [7]:
df_csv = spark.read.csv("./../sample_data/stock.csv",header=True,inferSchema=True)

In [8]:
df_csv.show()
df_csv.printSchema()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    